In [1]:
from local_tree_decomp import *
from local_hom_count import *
from local_hom_count_int import *
from local_helpers import *
from local_hom_count_int_pre import *

In [9]:
def speed_test(graph, target_graph, naive=True):
    basic_result = count_homomorphisms(graph, target_graph)
    int_rep_result = count_homomorphisms_int(graph, target_graph)
    better_int_rep_result = count_homomorphisms_int_pre(graph, target_graph)

    results = [basic_result, int_rep_result, better_int_rep_result]

    if naive:
        brute_force_result = len(enumerate_homomorphisms(graph, target_graph))
        results.append(brute_force_result)

    if all(result == better_int_rep_result for result in results):
        print(better_int_rep_result)

        if naive:
            print('\nBrute force:')
            %timeit len(enumerate_homomorphisms(graph, target_graph))
        
        print('\nBasic:')
        %timeit count_homomorphisms(graph, target_graph)
        
        print('\nInt representation:')
        %timeit count_homomorphisms_int(graph, target_graph)
    
        print('\nBetter int representation:')
        %timeit count_homomorphisms_int_pre(graph, target_graph)


In [13]:
graph = graphs.CycleGraph(4)
random_graph = graphs.RandomGNP(60, 0.5, seed=42)

speed_test(graph, random_graph, naive=False)


816368

Basic:
9.67 s ± 738 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Int representation:
3.65 s ± 18.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Better int representation:
758 ms ± 16.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [5]:
graph = graphs.CycleGraph(6)
target_graph = graphs.PetersenGraph()

speed_test(graph, target_graph)


990

Brute force:
12.7 s ± 452 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

Basic:
93.1 ms ± 4.77 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Int representation:
35.6 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

Better int representation:
13.5 ms ± 484 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
random_graph.size()

880